<small><i>December 2020 - This notebook was created by [Oriol Pujol Vila](http://www.maia.ub.es/~oriol). Source and [license](./LICENSE.txt) info are in the folder.</i></small>

# Unsupervised learning (in pytorch)

+ Autoencoders
+ Pretraining
+ Manifold learning
+ Sparse coding

In [2]:
#verify torch
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.5154, 0.7644, 0.4711],
        [0.3706, 0.8213, 0.5367],
        [0.0985, 0.8863, 0.9981],
        [0.9071, 0.8590, 0.3166],
        [0.6586, 0.3844, 0.0997]])


In [50]:
# Load data
import numpy as np
from sklearn.datasets import load_digits
data = load_digits()

idx = np.random.permutation(data.data.shape[0])
idx_train = idx[:-100]
idx_test = idx[-100:]

train = torch.from_numpy(data.data[idx_train,:]).float()
test = torch.from_numpy(data.data[idx_test,:]).float()

train_y = data.target[idx_train]
test_y = data.target[idx_test]

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class myfirstautoencoder(nn.Module):
    def __init__(self,**kwargs):
        super().__init__()
        self.d_input = kwargs["dim"]
        self.dr1 = nn.Dropout(p=0.2)
        self.e1 = nn.Linear(self.d_input,64)
        self.e2 = nn.Linear(64,32)
        self.e3 = nn.Linear(32,10)
        self.d1 = nn.Linear(10,32)
        self.d2 = nn.Linear(32,64)
        self.d3 = nn.Linear(64,self.d_input)
        
    def forward(self,x):
        x = self.dr1(x)
        x = F.relu(self.e1(x))
        x = F.relu(self.e2(x))
        x = self.bottleneck = F.relu(self.e3(x))
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        prediction = self.d3(x)
        return prediction
    

In [ ]:
net = myfirstautoencoder(dim = 64)
print(net)


In [ ]:
input = torch.randn(32,64)

out = net(input)
print(out)

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) 

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
iters = 15000


running_loss = 0.0
for i in range(iters):
    # get the inputs; data is a list of [inputs, labels]
    idx = np.random.randint(0,train.shape[0],size = 32)
    inputs = train[idx,:]
    labels = inputs

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 200 == 199:    # print every 200 iterations
        print('[%5d] loss: %.3f' %
              (i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

In [ ]:
# evaluate model:
net.eval()
k= 101

with torch.no_grad():
    reconstruction = net(train[k:k+1,:])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

t=train[k:k+1,:].numpy()
#r=reconstruction.detach().numpy()
r=reconstruction.numpy()
plt.figure()
plt.subplot(1,2,1)
plt.imshow(t.reshape((8,8)),cmap="gray")
plt.subplot(1,2,2)
plt.imshow(r.reshape((8,8)),cmap="gray")

# Inner Representations

In [ ]:
net.forward(train)
code = net.bottleneck

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(5,random_state=0)
clf.fit(code.detach().numpy(),train_y)

clf2 = RandomForestClassifier(5,random_state=0)
clf2.fit(train.detach().numpy(),train_y)

In [ ]:
net.forward(test)
code = net.bottleneck

print("Inner representation: " + str(clf.score(code.detach().numpy(),test_y)))
print("Original data: " + str(clf2.score(test.detach().numpy(),test_y)))

# Sparse dictionaries

In [ ]:
%reset -f

# Load data
import torch
import numpy as np
from sklearn.datasets import load_digits
data = load_digits()

idx = np.random.permutation(data.data.shape[0])
idx_train = idx[:-100]
idx_test = idx[-100:]

train = torch.from_numpy(data.data[idx_train,:]).float()
test = torch.from_numpy(data.data[idx_test,:]).float()

train_y = data.target[idx_train]
test_y = data.target[idx_test]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class myfirstautoencoder(nn.Module):
    def __init__(self,**kwargs):
        super().__init__()
        self.d_input = kwargs["dim"]
        self.dr1 = nn.Dropout(p=0.2)
        self.e1 = nn.Linear(self.d_input,64)
        self.dr2 = nn.Dropout(p=0.2)
        self.e2 = nn.Linear(64,128)
        self.e3 = nn.Linear(128,256)
        self.d1 = nn.Linear(256,128)
        self.d2 = nn.Linear(128,64)
        self.d3 = nn.Linear(64,self.d_input)
        
    def forward(self,x):
        x = self.dr1(x)
        x = F.relu(self.e1(x))
        x = self.dr2(x)
        x = F.relu(self.e2(x))
        bottleneck = F.relu(self.e3(x))
        x = F.relu(self.d1(bottleneck))
        x = F.relu(self.d2(x))
        prediction = self.d3(x)
        return prediction,bottleneck
    

In [ ]:
net = myfirstautoencoder(dim = 64)


In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net.train()

iters = 30000


running_loss = 0.0
for i in range(iters):
    # get the inputs; data is a list of [inputs, labels]
    idx = np.random.randint(0,train.shape[0],size = 32)
    inputs = train[idx,:]
    labels = inputs

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs,bn = net(inputs)
    loss = criterion(outputs, labels) + 0.001*torch.norm(bn,1)
    #loss += 0.00000001*torch.norm(bn,1)
    
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 200 == 199:    # print every 200 iterations
        print('[%5d] loss: %.3f' %
              (i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

In [ ]:
# evaluate model:
net.eval()
k= 101

with torch.no_grad():
    reconstruction,spar = net(train[k:k+1,:])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt



t=train[k:k+1,:].numpy()
r=reconstruction.detach().numpy()
plt.figure()
plt.subplot(1,2,1)
plt.imshow(t.reshape((8,8)),cmap="gray")
plt.subplot(1,2,2)
plt.imshow(r.reshape((8,8)),cmap="gray")

In [ ]:
plt.plot(spar[0])

# Manifolds

In [ ]:
# Load data
import numpy as np
from sklearn.datasets import load_digits
data = load_digits()

idx = np.random.permutation(data.data.shape[0])
idx_train = idx[:-100]
idx_test = idx[-100:]

train = torch.from_numpy(data.data[idx_train,:]).float()
test = torch.from_numpy(data.data[idx_test,:]).float()

train_y = data.target[idx_train]
test_y = data.target[idx_test]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class myfirstautoencoder(nn.Module):
    def __init__(self,**kwargs):
        super().__init__()
        self.d_input = kwargs["dim"]
        self.dr1 = nn.Dropout(p=0.2)
        self.e1 = nn.Linear(self.d_input,64)
        self.e2 = nn.Linear(64,32)
        self.e3 = nn.Linear(32,16)
        self.e4 = nn.Linear(16,2)
        self.d1 = nn.Linear(2,16)
        self.d2 = nn.Linear(16,32)
        self.d3 = nn.Linear(32,64)
        self.d4 = nn.Linear(64,self.d_input)
        
    def forward(self,x):
        x = self.dr1(x)
        x = F.relu(self.e1(x))
        x = F.relu(self.e2(x))
        x = F.relu(self.e3(x))
        x = bottleneck = F.relu(self.e4(x))
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        x = F.relu(self.d3(x))
        prediction = self.d4(x)
        return prediction,bottleneck
    

In [ ]:
net = myfirstautoencoder(dim = 64)
print(net)


In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net.train()

iters = 30000


running_loss = 0.0
for i in range(iters):
    # get the inputs; data is a list of [inputs, labels]
    idx = np.random.randint(0,train.shape[0],size = 32)
    inputs = train[idx,:]
    labels = inputs

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs,bn = net(inputs)
    loss = criterion(outputs, labels)
    #loss += 0.00000001*torch.norm(bn,1)
    
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 200 == 199:    # print every 200 iterations
        print('[%5d] loss: %.3f' %
              (i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

In [ ]:
# evaluate model:
net.eval()

with torch.no_grad():
    reconstruction,representation = net(train)


In [ ]:
representation = representation.numpy()

In [ ]:
from scipy.spatial import distance 
N=10
vx = np.linspace(np.min(representation[:,0]),np.max(representation[:,0]),N)
vy = np.linspace(np.min(representation[:,1]),np.max(representation[:,1]),N)

def is_visited(x,l):
    for item in l:
        if np.abs(x-item)<1e-10:
            return True
    return False
visited=[]
idx_mat=np.zeros((N,N))       
for i in range(N):
    for j in range(N):
        d = distance.cdist(np.array([vx[i],vy[j]])[np.newaxis,:], representation)
        idx_sort = np.argsort(d)[0]
        idx_not_visited=[tmp for tmp in idx_sort if not(is_visited(tmp,visited))]
        if len(idx_not_visited)>0:
            idx_mat[i,j] = idx_not_visited[0]
            visited.append(idx_not_visited[0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
f, axarr = plt.subplots(N, N)

xs=train.numpy()

for i in range(N):
    for j in range(N):
        axarr[i,j].imshow(xs[int(idx_mat[i,j]),:].reshape((8,8)),cmap='gray', interpolation='nearest')
f.set_size_inches(10,10)